In [1]:
from msmbuilder.dataset import dataset
from msmbuilder.msm import MarkovStateModel
import numpy as np
from msmbuilder.msm import implied_timescales
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from msmbuilder.msm import MarkovStateModel

/home/dongdong/software/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/dongdong/software/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
cd /home/dongdong/SCR/rat_xtc

/scratch/gpfs/dongdong/rat_xtc


In [96]:
xyz = dataset("./cluster/*.xtc",topology='./ykt6_begin.pdb',stride=5)

In [44]:
from msmbuilder.featurizer import ContactFeaturizer
featurizer = ContactFeaturizer()

In [45]:
contacts=featurizer.fit_transform(xyz)

In [6]:
len(contacts[0][0])

18528

In [46]:
from msmbuilder.decomposition import tICA
tica_model = tICA(lag_time=2, n_components=10)

In [47]:
tica_model = tica_model.fit(contacts)

In [48]:
tica_trajs = tica_model.transform(contacts)

In [49]:
from msmbuilder.cluster import MiniBatchKMeans
clusterer = MiniBatchKMeans(n_clusters=100, random_state=42)

In [50]:
clustered_trajs = clusterer.fit_transform(tica_trajs)

In [51]:
sample=[]
for cc in range(100):
    temp=[]
    for trj in range(len(clustered_trajs)):
        for tt in range(len(clustered_trajs[trj])):
            if cc ==clustered_trajs[trj][tt]:
                temp.append(trj)
                temp.append(tt)
                break
        continue
    #print cc
    #print temp
    sample.append(temp)

In [54]:
import mdtraj
import os
n=0
for ss in sample:
    temp = mdtraj.load_xtc(xyz.filename(ss[0]),top = './ykt6_begin.pdb',frame = int(ss[1]))
    pdb_name= './cluster/state'+ str(n) +'.pdb'
    temp.save_pdb(pdb_name)
    n=n+1

In [35]:
0 in clustered_trajs[0]

True

In [62]:
print tica_model.components_[0]

[-0.00382548  0.00209877 -0.00045398 ..., -0.00164505 -0.0061341
  0.00812572]


In [72]:
featurizer.

'closest-heavy'

In [75]:
from msmbuilder.decomposition import PCA
pca_model = PCA(n_components=4)
# fit and transform can be done in seperate steps:
pca_model = pca_model.fit(contacts)
pca_trajs = pca_model.transform(contacts)

In [76]:
pca_model.explained_variance_ratio_

array([ 0.43707001,  0.13032667,  0.11349556,  0.04900301])

In [79]:
np.max(pca_model.components_[0])

0.015892906659812609

In [94]:
len(contacts[0][0])

18528

In [102]:
featurizer.describe_features(t)[-1]

{'atominds': [[1735, 1737, 1738, 1739, 1740, 1741, 1744, 1745],
  [1760, 1762, 1763, 1764, 1765, 1766, 1767]],
 'featuregroup': 'closest-heavy',
 'featurizer': 'Contact',
 'otherinfo': '20',
 'resids': array([191, 194]),
 'resnames': ['ASN', 'CYSH'],
 'resseqs': [192, 195]}

In [98]:
import mdtraj as md
t=md.load('cluster/ffit5.xtc',top='ykt6_begin.pdb',stride=5)

In [106]:
pca1=pca_model.components_[0]
pca2=pca_model.components_[1]
tica1=tica_model.components_[0]
tica2=tica_model.components_[1]
wf=open('pca_tica.cs','w')
wf.write('#contact   pca1 pca2 tica1 tica2\n')
n=0
for i in range(1,193):
    for j in range(i+3,196):     
        wf.write(str(i)+'_'+str(j)+'    '+str(pca1[n])+'    '+str(pca2[n])+'    '+str(tica1[n])+'    '+str(tica2[n])+'\n')
        n+=1
wf.close()